In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb

In [3]:
wandb.init(project="mnist-model-evaluation", entity="kimkj")

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 1024)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)


wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)


In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)


64.8%

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%
100.0%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [7]:
wandb.watch(model, log="all")


[]

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            wandb.log({"train_loss": loss.item()})
            
for epoch in range(1, 5):
    train(epoch)


/home/kjk/.local/lib/python3.10/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
wandb: ERROR Error while calling W&B API: run kimkj/mnist-model-evaluation/et0c7tx4 was previously created and deleted; try a new run name (<Response [409]>)
/home/kjk/.local/lib/python3.10/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(testloader.dataset)
    accuracy = 100. * correct / len(testloader.dataset)
    wandb.log({"test_loss": test_loss, "accuracy": accuracy})

test()
